In [ ]:
from datetime import datetime, time
from typing import List, Dict, Any, Callable

import numpy as np
import pandas as pd
from nptyping import DataFrame, NDArray, Shape, Float

In [ ]:
noise_degree_standard: Dict[str, Dict[str, int]] = {
    '0类': {
        '昼间': 50,
        '夜间': 40,
    },
    '1类': {
        '昼间': 55,
        '夜间': 45,
    },
    '2类': {
        '昼间': 60,
        '夜间': 50,
    },
    '3类': {
        '昼间': 65,
        '夜间': 55,
    },
    '4类': {
        '昼间': 70,
        '夜间': 55,
    },
}

distr_dict: Dict[str, Callable] = {
    "正态分布": np.random.normal,
    "拉普拉斯分布": np.random.laplace,
    "逻辑分布": np.random.logistic,
    "耿贝尔分布": np.random.gumbel,
}


In [ ]:
class EnvNoise():
    '''随机环境噪声'''
    def __init__(
        self,
        in_df_distr: DataFrame,
        in_df_random: DataFrame,
        tag: str,
        noise_degree: str,
        noise_degree_standard: Dict[str, Dict[str, int]] = noise_degree_standard,
        distr_dict: Dict[str, Callable] = distr_dict,
        t_minute: int = 10,
        freq_weighting: str = 'A',
        count_ps: int = 2,
        time_weighting: str = 'F',
        ) -> None:
        '''定义'''
        self.noise_degree_standard: Dict[str, Dict[str, int]] = noise_degree_standard # 噪声等级标准
        self.distr_dict: Dict[str, Callable] = distr_dict # 分布方法字典
        self.in_df_distr: DataFrame = in_df_distr # 分布噪声数值df
        self.in_df_random: DataFrame = in_df_random # 随机噪声数值df
        self.tag: str = tag # 数据标签
        self.noise_degree: str = noise_degree # 噪声类别
        self.night_noise_limit: int = self.noise_degree_standard[self.noise_degree]['夜间'] # 夜间噪声限值
        self.t_minute: int = t_minute # 监测时长(min)
        self.freq_weighting: str = freq_weighting # 频率计权方式
        self.count_ps: int = count_ps # 每秒监测次数
        self.size: int = self.t_minute * self.count_ps * 60 # 监测时长的所有监测次数
        self.time_weighting: str = time_weighting # 时间计权方式
    
    def create_distr_noise_info_df(self, distr_name: str):
        '''从分布噪声数值df生成随机分布噪声数值相关信息df'''
        distr_noise_info_list: List[Any] = []
        for i in range(self.in_df_distr.shape[0]):
            current_datetime = self.in_df_distr.loc[i, '日期时间']
            r1: int = int(self.in_df_distr.loc[i, '监测范围1']) # type: ignore
            r2: int = int(self.in_df_distr.loc[i, '监测范围2']) # type: ignore
            leq: float = float(self.in_df_distr.loc[i, '等效连续声级Leq']) # type: ignore
            sd_val: float = float(self.in_df_distr.loc[i, '标准差SD']) # type: ignore
            row_noise_info = self.create_distr_noise_info_dict(
                current_datetime, # type: ignore
                r1,
                r2,
                leq,
                sd_val,
                distr_name
            )
            distr_noise_info_list.append(row_noise_info)
            distr_noise_info_df: DataFrame = pd.DataFrame(data=distr_noise_info_list)
            return distr_noise_info_df

    def create_distr_noise_info_dict(
        self,
        current_datetime: datetime,
        r1: int,
        r2: int,
        leq: float,
        sd_val: float,
        distr_name: str
    ) -> Dict[str, Any]:
        '''从分布噪声数值生成随机分布噪声数值相关信息'''
        noise_array: NDArray[Shape[self.size], Float] = (
            self
            .distr_dict[distr_name]
            (leq, sd_val, self.size)
        )
        noise_info: Dict[str, Any] = self.create_noise_info_dict(noise_array)
        diff_val: float = float(10.0 * np.log10(self.t_minute * 60)) # 暴露声级和等效声级之间的差
        sel: float = leq + diff_val
        noise_info['r1'] = r1
        noise_info['r2'] = r2
        noise_info['dt'] = current_datetime
        noise_info['leq'] = leq
        noise_info['sel'] = sel
        is_night: bool = (
            current_datetime >= datetime.combine(current_datetime.date(), time(22, 0, 0))
            or
            current_datetime <= datetime.combine(current_datetime.date(), time(6, 0, 0))
        )
        if is_night:
            if noise_info['lmax'] <= self.night_noise_limit + 15:
                noise_info['超过限值'] = False
            else:
                noise_info['超过限值'] = True
        else:
            noise_info['超过限值'] = None

        return noise_info

    def create_random_noise_info_df(self):
        '''从分布噪声数值df生成随机分布噪声数值相关信息df'''
        random_noise_info_list: List[Any] = []
        for i in range(self.in_df_random.shape[0]):
            current_datetime = self.in_df_distr.loc[i, '日期时间']
            r1: int = int(self.in_df_random.loc[i, '监测范围1']) # type: ignore
            r2: int = int(self.in_df_random.loc[i, '监测范围2']) # type: ignore
            min_val: int = int(self.in_df_random.loc[i, '随机值下限']) # type: ignore
            max_val: int = int(self.in_df_random.loc[i, '随机值上限']) # type: ignore
            row_noise_info = self.create_random_noise_info_dict(
                current_datetime, # type: ignore
                min_val,
                max_val,
                r1,
                r2,
            )
            random_noise_info_list.append(row_noise_info)
            random_noise_info_df: DataFrame = pd.DataFrame(data=random_noise_info_list)
            return random_noise_info_df

    def create_random_noise_info_dict(
        self,
        current_datetime: datetime,
        min_val: int,
        max_val: int,
        r1: int,
        r2: int,
    ) -> Dict[str, Any]:
        '''从分布噪声数值生成随机分布噪声数值相关信息'''
        noise_array: NDArray[Shape[self.size], Float] = np.random.randint(min_val * 10, max_val * 10, self.size) / 10
        noise_info: Dict[str, Any] = self.create_noise_info_dict(noise_array)
        diff_val: float = float(10.0 * np.log10(self.t_minute * 60)) # 暴露声级和等效声级之间的差
        leq: float = float(np.mean(noise_array))
        sel: float = leq + diff_val
        noise_info['r1'] = r1
        noise_info['r2'] = r2
        noise_info['dt'] = current_datetime
        noise_info['leq'] = leq
        noise_info['sel'] = sel
        is_night: bool = (
            current_datetime >= datetime.combine(current_datetime.date(), time(22, 0, 0))
            or
            current_datetime <= datetime.combine(current_datetime.date(), time(6, 0, 0))
        )
        if is_night:
            if noise_info['lmax'] <= self.night_noise_limit + 15:
                noise_info['超过限值'] = False
            else:
                noise_info['超过限值'] = True
        else:
            noise_info['超过限值'] = None

        return noise_info


    def create_noise_info_dict(self, noise_array: NDArray) -> Dict[str, float]:
        '''从噪声数值信息生成噪声数值相关信息'''            
        noise_info_dict: Dict[str, float] = {
            # 'lmax': float(noise_array.max().round(1)),
            # 'lmin': float(noise_array.min().round(1)), # 最小值和最大值
            # 'l5': float(np.percentile(noise_array, 5).round(1)),
            # 'l10': float(np.percentile(noise_array, 10).round(1)),
            # 'l50': float(np.percentile(noise_array, 50).round(1)),
            # 'l90': float(np.percentile(noise_array, 90).round(1)),
            # 'l95': float(np.percentile(noise_array, 95).round(1)),
            # 'sd': float(noise_array.std().round(1)),
            'lmax': float(noise_array.max()),
            'lmin': float(noise_array.min()), # 最小值和最大值
            'l5': float(np.percentile(noise_array, 95)),
            'l10': float(np.percentile(noise_array, 90)),
            'l50': float(np.percentile(noise_array, 50)),
            'l90': float(np.percentile(noise_array, 10)),
            'l95': float(np.percentile(noise_array, 5)),
            'sd': float(noise_array.std()),
        }
        return noise_info_dict

In [ ]:
in_df_distr = pd.DataFrame([{
    "日期时间": pd.to_datetime("2023-01-01 00:32:07"),
    "等效连续声级Leq": 60.0,
    "标准差SD": 1.0,
    '监测范围1': 33,
    '监测范围2': 133
}])

in_df_random = pd.DataFrame([{
    "日期时间": pd.to_datetime("2023-01-01 00:32:07"),
    "随机值下限": 40,
    "随机值上限": 65,
    '监测范围1': 33,
    '监测范围2': 133
}])

In [ ]:
env_noise = EnvNoise(
    in_df_distr = in_df_distr,
    in_df_random = in_df_random,
    tag = 'Test',
    noise_degree='2类',
    noise_degree_standard=noise_degree_standard,
)

In [ ]:
env_noise.night_noise_limit + 15

In [ ]:
env_noise.create_distr_noise_info_df('正态分布')

In [ ]:
env_noise.create_random_noise_info_df()